# VERSAO DE DESENVOLVIMENTO, 1.0

In [13]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json

In [2]:
os.chdir('C:\Repos\DS-Apis-Globo')

## Função para carregar o arquivo de dados de tempo disponível para anuncios

In [3]:
def read_inv(file_inv):
    #file_inv = '../Data/tvaberta_inventory_availability.csv(1).csv'
    dateparser_inv = lambda x: datetime.strptime(x, '%d/%m/%Y')
    sep_inv = ';'
    parsedates_inv = ['date']
    df_inv = pd.read_csv(file_inv,
                         sep=sep_inv,
                         parse_dates=parsedates_inv,
                         date_parser=dateparser_inv)
    df_inv['weekday'] = df_inv.date.dt.dayofweek
    # acrescentando colunas do outro conjunto de dados de audiencia media e prevista para realizar mesclagem dos conjuntos
    # valores sao inicializados com -1, caso alguma requisicao retorne com -1 em audiencia prevista significa que a data do registro correspondeu apenas à este conjunto df_inv.
    df_inv['average_audience'] = -1
    df_inv['predicted_audience'] = -1
    return df_inv

## Função para carregar o arquivo de dados de media historica de anuncios


In [4]:
def read_pro(file_pro):
    #file_pro = '../Data/tvaberta_program_audience(1).csv'
    dateparser_pro = lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ")
    sep_pro = ','
    parsedates_pro = ['program_start_time']
    indexcol_pro = 'program_start_time'
    df_pro = pd.read_csv(file_pro,
                         sep=sep_pro,
                         parse_dates=parsedates_pro,
                         date_parser=dateparser_pro)
    df_pro['exhibition_date'] = pd.to_datetime(df_pro['exhibition_date'], format='%Y-%m-%d')
    df_pro = df_pro.rename(columns={"exhibition_date": "date"})
    df_pro['weekday'] = df_pro.date.dt.dayofweek  
    # inicializando audiencia prevista em -1
    df_pro['predicted_audience'] = -1
    # valores de tempo estimado inicializados com -1, caso alguma requisicao retorne com -1 em tempo estimado significa que a data do registro correspondeu apenas à este conjunto df_pro.
    df_pro['available_time'] = -1
    del df_pro['program_start_time']
    return df_pro

## Calculo da mediana que recebe como parametro um pandas dataframe, uma coluna alvo de calculo e uma janela temporal de calculo.


In [5]:
def median_window(df,target,window):
    if len(df[target])>0:
        # o calculo da mediana se da pelo valor na posicao central do conjunto ordenado portanto as linhas do dataframe sao ordenadas com base na coluna alvo, com complexidade n*log(2)*n em qualquer caso de ordenacao. Dado que o pandas utiliza o algoritmo merge sort na funcao .sort_values
        df.sort_values(by=[target])
        # captura-se o primeiro valor da coluna alvo do daframe
        fill = df[target].values[0]
        # se o tamanho do dataframe for par, deve-se calcular a mediana atraves da media do valor central e o proximo elemento, dentro da janela de calculo
        if window % 2 == 0:
            s1 = int(window / 2)
            s2 = int(window / 2) + 1
            # para otimizacão do calculo da mediana sao utilizados 2 vetores com a coluna de valores deslocados pelo tamanho da janela de calculo, que no caso deste exercicio tem tamanho 4. Portanto a mediana, de janela 4, para qualquer registro dentro do dataframe ordenado sera a media aritmetica entre o 2o e 3o valor anterior à este registro.
            laux1 = df[target].shift(s1,fill_value=fill)
            laux2 = df[target].shift(s2,fill_value=fill)
            return ((laux1 + laux2) / 2)
        else:
            s1 = int(window/2) + 1
            return df[target].shift(s1,fill_value=fill)

# Função para varrer o dataframe com os registros de audiencia e calcular a mediana para cada registro pertencente a uma localizacao, codigo de programa e dia da semana distinto.

In [6]:
def df_predicted_audience(df):
    # obtendo lista de localizacoes distintas de transmissao
    l_signal = df.signal.unique()
    # iteracao para cada localizacao
    for v_signal in l_signal:
        # filtrando o dataframe para a localizacao da iteracao
        df_aux = df.loc[df['signal'] == v_signal]
        # obtendo lista de programas distintos dentro do dataframe filtrado
        l_program_code = df_aux.program_code.unique()
        # iterando sobre cada programa
        for v_program in l_program_code:
            # filtrando o dataframe a localizacao e codigo de programa da iteracao
            df_aux = df.loc[(df['signal'] == v_signal) & (df['program_code'] == v_program)]
            # obtendo lista de dias da semana disponiveis dentro do dataframe filtrado
            l_weekdays = df_aux.weekday.unique()
            for v_day in l_weekdays:
                # obtendo um dataframe auxiliar com todos os registros da localizacao, codigo de programa e dia da semana filtrados
                df_aux = df.loc[(df['weekday'] == v_day) & (df['signal'] == v_signal) & (
                            df['program_code'] == v_program)]
                #df_aux.sort_values(by=['average_audience'])
                # acionando a funcao para o calculo da audiencia estimada por meio da mediana, dentro do dataframe auxiliar
                df_aux['predicted_audience'] = median_window(df_aux,'average_audience',4)
                # gravando os valores de audiencia estimada correspondentes aos indices dos registros filtrados no dataframe resultante.
                df.predicted_audience.loc[df_aux.index] = df_aux['predicted_audience'].values
    return df

## Carregando conjunto de dados

In [7]:
df_inv = read_inv('./Data/tvaberta_inventory_availability.csv(1).csv')
df_pro = read_pro('./Data/tvaberta_program_audience.csv')

## Preparando conjunto resultante

In [8]:
df_res = df_pro.copy()
df_res = df_predicted_audience(df_res)
df_res

pis-Globo\env\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-6-c14b080747dd>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aux['predicted_audience'] = median_window(df_aux,'average_audience',4)
C:\Repos\DS-Apis-Globo\env\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,signal,program_code,date,average_audience,weekday,predicted_audience,available_time
0,SP1,PTV1,2020-06-05,9.338920e+05,4,9.338920e+05,-1
1,SP1,VALE,2020-06-24,1.835375e+06,2,1.835375e+06,-1
2,BH,PTV1,2020-05-23,2.874812e+05,5,2.874812e+05,-1
3,DF,AUTO,2020-05-03,8.649481e+04,6,8.649481e+04,-1
4,DF,FGGE,2020-05-03,1.521656e+05,6,1.521656e+05,-1
...,...,...,...,...,...,...,...
8982,NAC,N18H,2020-05-04,2.093098e+07,0,2.018653e+07,-1
8983,NAC,JHOJ,2020-05-13,1.331286e+07,2,1.318622e+07,-1
8984,NAC,SHOV,2020-06-07,8.610939e+06,6,9.232443e+06,-1
8985,NAC,SHOV,2020-05-17,1.056899e+07,6,8.947505e+06,-1


## Mesclando conjunto de audiencia e tempo disponivel por anuncio

In [9]:
df_res = pd.concat([df_res,df_inv])
df_res

,signal,program_code,date,average_audience,weekday,predicted_audience,available_time
0,SP1,PTV1,2020-06-05,9.338920e+05,4,9.338920e+05,-1
1,SP1,VALE,2020-06-24,1.835375e+06,2,1.835375e+06,-1
2,BH,PTV1,2020-05-23,2.874812e+05,5,2.874812e+05,-1
3,DF,AUTO,2020-05-03,8.649481e+04,6,8.649481e+04,-1
4,DF,FGGE,2020-05-03,1.521656e+05,6,1.521656e+05,-1
...,...,...,...,...,...,...,...
2287,BH,NBRA,2020-08-24,-1.000000e+00,0,-1.000000e+00,218
2288,BH,NBRA,2020-08-25,-1.000000e+00,1,-1.000000e+00,66
2289,BH,NBRA,2020-08-26,-1.000000e+00,2,-1.000000e+00,44
2290,BH,NBRA,2020-08-27,-1.000000e+00,3,-1.000000e+00,190


## Funções da REST-API

In [10]:
def get_predicted(_program_code,_date):
    return json.loads(df_res.loc[(df_res['date'] == _date) & (df_res['program_code'] == _program_code)][['available_time','predicted_audience']].to_json(orient="records"))

In [11]:
def get_period(begin,end):
    return json.loads(df_res.loc[(df_res['date'] >= begin) & (df_res['date'] <= end)][['program_code','available_time','predicted_audience']].to_json(orient="records"))

In [14]:
get_predicted('PTV1','2020-06-05')

[{'available_time': -1, 'predicted_audience': 933891.9638554216},
 {'available_time': -1, 'predicted_audience': 211398.8072289157},
 {'available_time': -1, 'predicted_audience': 997945.0060240964},
 {'available_time': -1, 'predicted_audience': 208046.9723311852},
 {'available_time': -1, 'predicted_audience': 285912.9795460913},
 {'available_time': -1, 'predicted_audience': 12325816.445783133}]

In [15]:
get_period('2020-05-03','2020-06-24')

 'SAME',
  'available_time': -1,
  'predicted_audience': 423477.6382978723},
 {'program_code': 'PTV1',
  'available_time': -1,
  'predicted_audience': 211398.8072289157},
 {'program_code': 'GLCO',
  'available_time': -1,
  'predicted_audience': 63191.4814814815},
 {'program_code': 'SHOV',
  'available_time': -1,
  'predicted_audience': 229360.7359243698},
 {'program_code': 'MEST',
  'available_time': -1,
  'predicted_audience': 474733.9605263158},
 {'program_code': 'BPRA',
  'available_time': -1,
  'predicted_audience': 78773.5033557047},
 {'program_code': 'TARA',
  'available_time': -1,
  'predicted_audience': 319404.2469765684},
 {'program_code': 'COR1',
  'available_time': -1,
  'predicted_audience': 413771.6666666667},
 {'program_code': 'JHOJ',
  'available_time': -1,
  'predicted_audience': 276943.9945054945},
 {'program_code': 'SAME',
  'available_time': -1,
  'predicted_audience': 423477.6382978723},
 {'program_code': 'PTV2',
  'available_time': -1,
  'predicted_audience': 19076